In [ ]:
#library installation required, only necessary to install once
!pip install discord
!pip install nest_asyncio
!pip install neuralintents

In [ ]:
#import modules 
import discord
import os
import nest_asyncio 
import requests
import json
import csv
import random
import numpy as np
import pandas as pd 
import re # regular expression
import nltk
import requests
import pickle
import os
import string
import datetime as dt
from urllib.request import urlopen
from neuralintents import GenericAssistant
nest_asyncio.apply()

In [ ]:
#Tain nlp model with data 
import json
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

# Load JSON data
def preprocess_data():
    with open('DatabaseIn_jan24.json', 'r') as file:
        data = json.load(file)

    lemmatizer = WordNetLemmatizer()
    
    # Extract patterns and tags from JSON data
    X = []  # Patterns
    y = []  # Tags

    for intent in data['intents']:
        for pattern in intent['patterns']:
            X.append(pattern)
            y.append(intent['tag'])

    X_lemmatized = []
    for pattern in X:
        tokens = nltk.word_tokenize(pattern.lower())
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        X_lemmatized.append(' '.join(lemmatized_tokens))

    return X_lemmatized, y


In [ ]:
#nlp extracts the user input and predicts the tag
def predict_tag(input_text):
    # Preprocess the input text (tokenize and lemmatizer)
    lemmatizer = WordNetLemmatizer()
    test_lemmatized = ' '.join([lemmatizer.lemmatize(token.lower()) for token in nltk.word_tokenize(input_text)])
    
    # Preprocess the data
    X, y = preprocess_data()
    
    # Vectorize the preprocessed text data
    vectorizer = CountVectorizer()
    X_bow = vectorizer.fit_transform(X)
    
    # Train the Naive Bayes classifier
    model = MultinomialNB()
    model.fit(X_bow, y)
    
    # Transform the input text into Bag of Words representation
    test_bow = vectorizer.transform([test_lemmatized])
    
    # Predict the tag
    predictions = model.predict(test_bow)
    
    return predictions[0]  # Return the first prediction since we are handling only one sentence

In [ ]:
#neutralintents
from neuralintents.assistants import BasicAssistant
chatbot = BasicAssistant('DatabaseIn_jan24.json')
chatbot.fit_model(epochs=50)
chatbot.save_model()

In [ ]:
#Calculate the celsius, fahreheit
def kelvin_to_Celsius_fahrenheit(kelvin):
    celsius = kelvin - 273.15
    fahrenheit = celsius * (9/5) + 32
    return celsius, fahrenheit

# Connect to the openweather website API (weather for Penang)
def get_weather():
    weatherBaseURL = "https://api.openweathermap.org/data/2.5/weather?"
    API_key = 'INSERT API KEY'
    CITY = "penang"
    weatherURL = weatherBaseURL + "appid=" + API_key + "&q=" + CITY
    weatherResponse = requests.get(weatherURL).json()

    # Weather data extracted from the openweather website API
    if 'main' in weatherResponse:
        tempKelvin = weatherResponse ['main']['temp']
        tempCelsius, tempFah = kelvin_to_Celsius_fahrenheit(tempKelvin)
        feelLikeKelvin = weatherResponse ['main']['feels_like']
        feelLikeCelsius, feelLikeFah = kelvin_to_Celsius_fahrenheit(feelLikeKelvin)
        windSpeed = weatherResponse['wind']['speed']
        humidity = weatherResponse['main']['humidity']
        description = weatherResponse['weather'][0]['description']
        sunriseTime = dt.datetime.utcfromtimestamp(weatherResponse['sys']['sunrise']+ weatherResponse['timezone'])
        sunsetTime = dt.datetime.utcfromtimestamp(weatherResponse['sys']['sunset']+ weatherResponse['timezone'])
        return tempCelsius, tempFah, feelLikeCelsius, feelLikeFah, description, windSpeed, humidity
        # return weather information
    else:
        return None

In [ ]:
# use weather api to check whether it is raining
def is_raining():
    url = "https://api.openweathermap.org/data/2.5/forecast?"
    API_key = 'INSERT API KEY'
    CITY = "penang"
    weatherURL = url + "appid=" + API_key + "&q=" + CITY
    weatherResponse = requests.get(weatherURL).json()
    
    if 'list' in weatherResponse:
        for forecast in weatherResponse['list']:
            if 'rain' in forecast.get('weather', {})[0].get('main', '').lower():
                return True
            else:
                return False
    else:
        return None

In [ ]:
#get the current time
def action_time():
    return dt.datetime.now().time().strftime('%H:%M')


In [ ]:
#get the real time plant information
def get_plant_data():
    # Load json file from plant system
    url = "https://cai3013jan24-default-rtdb.firebaseio.com/.json"
    response = requests.get(url)
    if response.status_code == 200:
        plant_data = pd.DataFrame.from_dict(response.json())
        humidity_value = plant_data['Humidity'].iloc[0]  
        light_level_value = plant_data['LightLevel'].iloc[0]
        temperature_value = plant_data['Temperature'].iloc[0]
        return humidity_value, light_level_value, temperature_value
    else:
        return None

In [ ]:
#use fuzzy logic to get plant health condition
# import necessary libraries
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from skfuzzy import control as ctrl

def evaluate_plant_health(humidity, light_level, temperature):
    # Create Antecedent/Consequent objects to hold universe variables and membership functions      
    Humidity = ctrl.Antecedent(np.arange(0, 101, 1), 'Humidity')
    LightLevel   = ctrl.Antecedent(np.arange(0, 101, 1), 'LightLevel')
    Temperature   = ctrl.Antecedent(np.arange(0, 101, 1), 'Temperature')
    Plant_health = ctrl.Consequent(np.arange(0, 101, 1), 'Plant_health')

    # Define membership functions for Humidity
    Humidity['low'] = fuzz.trapmf(Humidity.universe, [0, 0, 20, 40])
    Humidity['medium'] = fuzz.trimf(Humidity.universe, [30, 50, 100])
    Humidity['high'] = fuzz.trimf(Humidity.universe, [60, 100, 100])
    # Define membership functions for LightLevel
    LightLevel['low'] = fuzz.trimf(LightLevel.universe, [0, 10, 30])
    LightLevel['medium'] = fuzz.trimf(LightLevel.universe, [20, 50, 80])
    LightLevel['high'] = fuzz.trimf(LightLevel.universe, [70, 90, 100])
    # Define membership functions for Temperature
    Temperature['low'] = fuzz.trimf(Temperature.universe, [0, 5, 15])
    Temperature['medium'] = fuzz.trimf(Temperature.universe, [10, 20, 35])
    Temperature['high'] = fuzz.trimf(Temperature.universe, [30, 60, 100])
    # Define membership functions for Plant health
    Plant_health['bad'] = fuzz.trimf(Plant_health.universe, [0, 0, 40])
    Plant_health['decent'] = fuzz.trimf(Plant_health.universe, [30, 55, 70])
    Plant_health['good'] = fuzz.trimf(Plant_health.universe, [65, 100, 100])

    #Define rules
    rule1 = ctrl.Rule(Humidity['low'] | LightLevel['low'] | Temperature['high'], Plant_health['bad'])
    rule2 = ctrl.Rule(Humidity['medium'] | LightLevel['medium'] | Temperature['medium'], Plant_health['decent'])
    rule3 = ctrl.Rule(Humidity['high'] | LightLevel['high'] | Temperature['medium'], Plant_health['good'])

    #control system
    health_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
    plant = ctrl.ControlSystemSimulation(health_ctrl)
     
    plant.input['Humidity'] = humidity
    plant.input['LightLevel'] = light_level
    plant.input['Temperature'] = temperature

    # Compute output
    plant.compute()
     # Return the computed plant health value
    return plant.output['Plant_health']
     

In [ ]:
#use fuzzy logic to identify if plant is require watering
def plant_water_require(humidity, tempCelsius, windSpeed):
    # Create Antecedent/Consequent objects to hold universe variables and membership functions
    tempCelsius_an = ctrl.Antecedent(np.arange(0, 101, 1), 'main_temp')
    windSpeed_an   = ctrl.Antecedent(np.arange(0, 101, 1), 'wind_speed')
    Humidity_an   = ctrl.Antecedent(np.arange(0, 101, 1), 'main_humidity')
    Require_water_an = ctrl.Consequent(np.arange(0, 101, 1), 'Require_water')

     # Define membership functions for Humidity
    Humidity_an['low'] = fuzz.trapmf(Humidity_an.universe, [0, 0, 1, 30])
    Humidity_an['medium'] = fuzz.trimf(Humidity_an.universe, [0, 45, 100])
    Humidity_an['high'] = fuzz.trapmf(Humidity_an.universe, [70, 70, 100,100])
     # Define membership functions for WindSpeed
    windSpeed_an['low'] = fuzz.trimf(windSpeed_an.universe, [0, 15, 30])
    windSpeed_an['medium'] = fuzz.trimf(windSpeed_an.universe, [20, 40, 60])
    windSpeed_an['high'] = fuzz.trimf(windSpeed_an.universe, [50, 75, 100])
     # Define membership functions for Temperature
    tempCelsius_an['low'] = fuzz.trimf(tempCelsius_an.universe, [0, 5, 15])
    tempCelsius_an['medium'] = fuzz.trimf(tempCelsius_an.universe, [10, 20, 35])
    tempCelsius_an['high'] = fuzz.trimf(tempCelsius_an.universe, [30, 60, 100])
    # Define membership functions for Water requirement
    Require_water_an['yes'] = fuzz.trimf(Require_water_an.universe, [0, 0, 40])
    Require_water_an['maybe'] = fuzz.trimf(Require_water_an.universe, [30, 48, 70])
    Require_water_an['no'] = fuzz.trimf(Require_water_an.universe, [59, 100, 100])
    
    #Define rules
    rule1 = ctrl.Rule(Humidity_an['low'] | windSpeed_an['high'] | tempCelsius_an['high'], Require_water_an['yes'])
    rule2 = ctrl.Rule(Humidity_an['medium'] | windSpeed_an['medium'] | tempCelsius_an['medium'], Require_water_an['maybe'])
    rule3 = ctrl.Rule(Humidity_an['high'] | windSpeed_an['low'] | tempCelsius_an['medium'], Require_water_an['no'])
    #Control system
    water_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
    water = ctrl.ControlSystemSimulation(water_ctrl)
    
    water.input['main_humidity'] = humidity
    water.input['wind_speed'] = windSpeed
    water.input['main_temp'] = tempCelsius
    #compute output
    water.compute()
    #Return the computed water requirement 
    return water.output['Require_water']

In [ ]:
# Discord token for bot authentication
TOKEN = 'INSERT TOKEN'
client = discord.Client(intents=discord.Intents.all())

In [ ]:
#Main body of chatbot
class MyClient(discord.Client):
    
    
    async def on_ready(self):
        print(f'Logged in as {self.user} (ID: {self.user.id})')
        print('------')
        
    
    async def on_message(self, message):
        # To prevent the bot from replying to itself
        if message.author.id == self.user.id:
            return
        

        # While the bot is waiting on a response from the model
        # set its status as typing for user-friendliness
        async with message.channel.typing():  
            
            # NLP to extract user input and predict tag  
            tag = predict_tag(message.content)
             
            
            if tag.startswith('greeting'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                 
            if tag.startswith('goodbye'):
                response = chatbot.process_input(tag)
                await message.channel.send(response)
                
            if tag.startswith('thanks'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                
            if tag.startswith('funny'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
            
            if tag.startswith('name'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
            
            if tag.startswith('lifeTime'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                
            if tag.startswith('times'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                await message.channel.send(action_time())
            
            if tag.startswith('plantidentities'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                
            if tag.startswith('IndoorPlant'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                
            if tag.startswith('OutdoorPlant'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                
            if tag.startswith('moreWaterPlant'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                
            if tag.startswith('normalWaterPlant'):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                
            if tag.startswith("weather"):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                weather_data = get_weather()
                tempCelsius = weather_data[0]
                tempFah = weather_data[1]
                feelLikeCelsius = weather_data[2]
                feelLikeFah = weather_data[3]
                description = weather_data[4]
                windSpeed = weather_data[5]
                humidity = weather_data[6]
                res = (f"Current Temperature: {tempCelsius:.2f}°C / {tempFah:.2f}°F\n"
                                f"Feels Like Temperature: {feelLikeCelsius:.2f}°C / {feelLikeFah:.2f}°F\n"
                       f"General Weather: {description}\n Wind Speed: {windSpeed}m/s\n Humidity: {humidity}%")     
                await message.channel.send(res)
           
            if tag.startswith("feeling"):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                
                plantData = get_plant_data()
                if plantData is not None:
                    humidity, light_level, temperature = plantData
                    plant_health = evaluate_plant_health(humidity, light_level, temperature)
                    if plant_health <=40:
                        res1 = ("I'm not feeling quite happy because I'm kinda sick. ˙◠˙It's getting warm in here! My skin feels dry, and the surroundings seem rather shadowy; I might need to turn on some lights.")
                    if plant_health<=70:
                        res1 = ("I'm feeling okay, with a balanced level of humidity and not too dry. I'm feeling comfortable, and the lighting is moderate, providing a decent amount of illumination.")
                    else:
                        res1 = ("I'm feeling happy and great! \°'U'° / It's quite humid in here, almost like a sauna! Despite that, I feel comfortable. It's as bright as day, with the sun shining inside!")    
                    await message.channel.send(res1)

            if tag.startswith("raining"):
                response = chatbot.process_input(message.content)
                await message.channel.send(response)
                weather = get_weather()
                
                if weather is not None:
                    tempCelsius = weather[0]
                    windSpeed = weather[5]
                    humidity = weather[6]
                    water = plant_water_require(humidity, tempCelsius, windSpeed)
                    TrueOrFalse = is_raining()
                    if water <=70 and TrueOrFalse =="True":
                       res2 =("No, It is going to rain later.")
                    if water <=70 and TrueOrFalse =="False":
                       res2 =("Yes, please provide me some water. It is not going to rain later.")
                    else:
                        if TrueOrFalse =="False":
                           res2 =("Yes, I need some water, it's not going to rain later.)")
                        else:
                            TrueOrFalse =="True"
                            res2 = ("No, even later is going to rain, but I'm kinda full of water.")
                    await message.channel.send(res2)


# Set up the intents to allow certain events to be received by the chatbot  
intents = discord.Intents.default()
intents.message_content = True

def main():
  client = MyClient(intents=intents)
    # Run the bot with the provided token
  client.run(TOKEN)


if __name__ == '__main__':
    # Call the main function if the script is being executed 
  main()
